In [2]:
import pandas as pd
import numpy as np

In [64]:
data = pd.read_csv("../data/processed/period/ecmen/6_16/ecmenperiod2.csv.gz",compression='gzip',parse_dates=["Trade_Datetime","ecmen_pulicate_time"],index_col=["Trade_Datetime"])

In [65]:
weather_data = pd.read_csv("../data/processed/WeatherData/ecmen_weather_subclass.csv", parse_dates=["Trans_INIT_Time"])

In [66]:
data.head()

,Contract_Delivery_Date,Vwap,ecmen_pulicate_time,dst_flag,Normal_Vwap
Trade_Datetime,,,,,
2015-01-28 13:41:00,1503,2.860,2015-01-28 13:40:00,0,1.000
2015-01-28 13:42:00,1503,2.858,2015-01-28 13:40:00,0,0.999
2015-01-28 13:43:00,1503,2.855,2015-01-28 13:40:00,0,0.998
2015-01-28 13:44:00,1503,2.852,2015-01-28 13:40:00,0,0.997
2015-01-28 13:45:00,1503,2.853,2015-01-28 13:40:00,0,0.998


In [67]:
weather_data.head()

,Trans_INIT_Time,VALUE_hdd,10Y_NORMAL_hdd,Sum_Value_hdd,Delta_Full_hdd,Delta_Sub_hdd,VALUE_cdd,10Y_NORMAL_cdd,Sum_Value_cdd,Delta_Full_cdd,Delta_Sub_cdd
0,2015-01-01 01:40:00,34.32,28.51,451.82,0.00,0.00,0.19,0.19,2.70,0.00,0.00
1,2015-01-01 13:40:00,31.67,28.67,444.46,-2.95,-7.40,0.33,0.17,2.55,-0.09,0.02
2,2015-01-02 01:40:00,30.88,28.67,453.45,8.99,10.39,0.31,0.17,2.67,0.12,0.04
3,2015-01-02 13:40:00,27.33,28.70,448.02,-4.52,-4.38,0.49,0.16,2.59,0.10,0.11
4,2015-01-03 01:40:00,27.17,28.70,462.12,14.10,7.93,0.47,0.16,2.42,-0.17,-0.19


In [68]:
res = data.groupby(['Contract_Delivery_Date', pd.Grouper(freq='D')]).agg(ecmen_public_time=('ecmen_pulicate_time', 'first'),dst_flag=('dst_flag', 'first'), Normal_vwap=('Normal_Vwap', 'last'))

In [69]:
merge_data = pd.merge(res, weather_data, how="inner",left_on='ecmen_public_time', right_on='Trans_INIT_Time',
                      suffixes=('_hdd', '_cdd'))
merge_data

,ecmen_public_time,dst_flag,Normal_vwap,Trans_INIT_Time,VALUE_hdd,10Y_NORMAL_hdd,Sum_Value_hdd,Delta_Full_hdd,Delta_Sub_hdd,VALUE_cdd,10Y_NORMAL_cdd,Sum_Value_cdd,Delta_Full_cdd,Delta_Sub_cdd
0,2015-01-28 13:40:00,0,1.009,2015-01-28 13:40:00,28.30,29.74,462.94,7.04,1.60,0.01,0.22,0.82,-0.41,-0.31
1,2015-01-29 13:40:00,0,1.051,2015-01-29 13:40:00,31.65,29.67,469.98,6.93,-0.13,0.03,0.23,0.94,-0.17,-0.19
2,2015-01-30 13:40:00,0,1.016,2015-01-30 13:40:00,32.57,29.61,439.15,-11.06,-8.19,0.03,0.24,0.72,-0.34,-0.29
3,2015-02-02 13:40:00,0,0.993,2015-02-02 13:40:00,36.06,29.32,450.47,15.91,11.13,0.02,0.27,0.34,-0.50,-0.43
4,2015-02-03 13:40:00,0,0.977,2015-02-03 13:40:00,32.15,29.19,439.76,12.23,9.49,0.11,0.29,0.48,-0.32,-0.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,2021-03-08 13:40:00,0,1.011,2021-03-08 13:40:00,15.83,21.28,277.39,-3.45,6.18,0.09,0.59,5.25,0.51,-0.01
545,2021-03-09 13:40:00,0,1.005,2021-03-09 13:40:00,15.14,21.03,276.29,-0.34,-4.76,0.41,0.60,5.40,-0.68,-0.75
546,2021-03-10 13:40:00,0,0.987,2021-03-10 13:40:00,15.02,20.71,273.50,1.04,-4.79,0.70,0.60,5.21,-0.59,-0.41
547,2021-03-11 13:40:00,0,1.009,2021-03-11 13:40:00,18.07,20.47,277.47,-0.27,-1.07,0.77,0.61,5.26,-0.65,-0.72


In [70]:
merge_data["Month"] = merge_data["Trans_INIT_Time"].dt.month

In [71]:
merge_data.drop(columns=['Trans_INIT_Time','dst_flag'],inplace=True)

In [72]:
merge_data

,ecmen_public_time,Normal_vwap,VALUE_hdd,10Y_NORMAL_hdd,Sum_Value_hdd,Delta_Full_hdd,Delta_Sub_hdd,VALUE_cdd,10Y_NORMAL_cdd,Sum_Value_cdd,Delta_Full_cdd,Delta_Sub_cdd,Month
0,2015-01-28 13:40:00,1.009,28.30,29.74,462.94,7.04,1.60,0.01,0.22,0.82,-0.41,-0.31,1
1,2015-01-29 13:40:00,1.051,31.65,29.67,469.98,6.93,-0.13,0.03,0.23,0.94,-0.17,-0.19,1
2,2015-01-30 13:40:00,1.016,32.57,29.61,439.15,-11.06,-8.19,0.03,0.24,0.72,-0.34,-0.29,1
3,2015-02-02 13:40:00,0.993,36.06,29.32,450.47,15.91,11.13,0.02,0.27,0.34,-0.50,-0.43,2
4,2015-02-03 13:40:00,0.977,32.15,29.19,439.76,12.23,9.49,0.11,0.29,0.48,-0.32,-0.38,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
544,2021-03-08 13:40:00,1.011,15.83,21.28,277.39,-3.45,6.18,0.09,0.59,5.25,0.51,-0.01,3
545,2021-03-09 13:40:00,1.005,15.14,21.03,276.29,-0.34,-4.76,0.41,0.60,5.40,-0.68,-0.75,3
546,2021-03-10 13:40:00,0.987,15.02,20.71,273.50,1.04,-4.79,0.70,0.60,5.21,-0.59,-0.41,3
547,2021-03-11 13:40:00,1.009,18.07,20.47,277.47,-0.27,-1.07,0.77,0.61,5.26,-0.65,-0.72,3


In [73]:
file_name = "ecmenperiod2"+".csv"
merge_data.to_csv(file_name,index=False, float_format='%.3f')